In [17]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import spacy
from sklearn.feature_extraction.text import CountVectorizer
%autosave 10
from wordcloud import WordCloud

Autosaving every 10 seconds


In [18]:
df=pd.read_csv("/home/a25misra/nyt_headline/combined.csv")
print(df.columns)
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head(5)

Index(['Unnamed: 0', 'abstract', 'pub_date', 'type_of_material', 'word_count',
       'print_page', 'headline', 'section_name', 'subsection_name'],
      dtype='object')


,abstract,pub_date,type_of_material,word_count,print_page,headline,section_name,subsection_name
0,"New York, New Jersey, Pennsylvania, Delaware, ...",2020-05-03T13:10:38+0000,News,1773,8.0,Seven States to Coordinate on Amassing Medical...,New York,NaN
1,"On our daily walks together, my young daughter...",2020-05-01T19:30:09+0000,Op-Ed,1481,10.0,The Courage to Be Alone,Opinion,Sunday Review
2,The small-scale events could create a road map...,2020-05-01T05:00:06+0000,News,1279,8.0,Tennis Coming Back Slowly With Exhibition Matches,Sports,Tennis
3,"By ordering in, a political reporter continues...",2020-05-01T18:24:58+0000,News,1175,3.0,The Best Part of the Campaign Trail (the Food!...,Food,NaN
4,You or your loved one is another year older no...,2020-05-02T16:00:07+0000,News,1331,7.0,"Go Ahead, Blow Out the Candles on Zoom",Smarter Living,NaN


In [19]:
df.isna().sum()

abstract             227
pub_date               0
type_of_material     227
word_count             0
print_page          2468
headline               0
section_name           0
subsection_name     3874
dtype: int64

In [20]:
print(len(df))
df=df[df.section_name.isin(['New York', 'Opinion', 'Sports','Your Money', 'World','Science', 'Business Day','Today’s Paper', 'U.S.', 'Technology',
       'Reader Center', 'Health','Sunday Review', 'Real Estate', 'Briefing',
       'Climate','Times Insider'])]

6363


In [21]:
nlp=spacy.load("en_core_web_sm")
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [22]:
df.pub_date=pd.to_datetime(df.pub_date)

In [23]:
df.date=df.pub_date.dt.date

/home/a25misra/nlp_proc/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [24]:
df_before=df[df.date<pd.to_datetime("2020-05-25")]
df_after=df[df.date>pd.to_datetime("2020-05-25")]

# Before

In [25]:
bow_before=pd.DataFrame(0.0, index=np.arange(len(df_before)),columns=[])
idx=0
for headline in df_before.headline:
    head=nlp(headline.lower())
    for token in head:
        if(token.is_stop ==False and token.is_punct == False):
            if(token.lemma_ not in bow_before.columns):
                bow_before[token.lemma_]=[0] * len(bow_before)
            bow_before[token.lemma_].iloc[idx]+=1
    idx+=1   
bow_before.head(5)

,seven,state,coordinate,amass,medical,equipment,fight,coronavirus,courage,tennis,...,uncharted,territory,italians,patriotic,tear,resist,restrict,felon,unconstitutional,shield
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
for col in bow_before.columns:
    if(col.isdigit() or col in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday'] ):
        bow_before.drop([col],inplace=True, axis=1)
    

In [27]:
'''
bow_before["date_data"]=df_before.pub_date.dt.date
"date_data" in bow_before.columns.tolist()
'''

'\nbow_before["date_data"]=df_before.pub_date.dt.date\n"date_data" in bow_before.columns.tolist()\n'

In [28]:
idf_before={}
for a in bow_before.columns:
    idf_before[a]=np.log(len(bow_before)/(len(bow_before[bow_before[a]!=0])+1))
print(idf_before)

{'seven': 6.824373670043086, 'state': 3.853959204473385, 'coordinate': 6.824373670043086, 'amass': 6.824373670043086, 'medical': 5.438079308923196, 'equipment': 6.824373670043086, 'fight': 4.5217885770490405, 'coronavirus': 2.114843468730752, 'courage': 6.418908561934922, 'tennis': 5.908082938168931, 'come': 4.150225020616557, 'slowly': 6.418908561934922, 'exhibition': 6.824373670043086, 'match': 6.824373670043086, 'philanthropy': 6.824373670043086, 'race': 4.878463520987773, 'factor': 6.824373670043086, 'get': 4.426478397244716, 'study': 4.809470649500821, 'show': 5.032614200815031, 'kim': 6.131226489483141, 'jong': 6.131226489483141, 'un': 6.131226489483141, 'happen': 4.259424312581549, 'go': 4.1853163404278275, 'lab': 5.725761381374976, 'u.s': 3.492169159867882, 'join': 6.418908561934922, 'federal': 5.214935757608986, 'initiative': 6.824373670043086, 'genome': 6.824373670043086, 'business': 4.150225020616557, 'burps': 6.824373670043086, 'scientist': 5.032614200815031, 'smell': 6.824

In [29]:
tfidf_before=bow_before.copy()
tfidf_before["containing_term"]=tfidf_before.gt(0).sum(axis=1)
print(len(tfidf_before[tfidf_before["containing_term"]==0]))
tfidf_before=tfidf_before.iloc[:,:].div(tfidf_before.containing_term,axis=0)
for key in idf_before.keys():
    tfidf_before.loc[:,key]=tfidf_before.loc[:,key]*idf_before[key]
tfidf_before.drop(["containing_term"],axis=1, inplace=True)
tfidf_before.dropna(how='any',axis=0,inplace=True)

1


In [30]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(tfidf_before)
print(svd.singular_values_)
terms = tfidf_before.columns
print(svd.explained_variance_)
for i, comp in enumerate(svd.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    topic=""
    for t in sorted_terms:
        topic=topic+" "+t[0]
    print(topic)

[9.84796125 9.11725796 9.08605256 7.96752961 7.66444176]
[0.04784847 0.0447326  0.04434991 0.02600336 0.02919077]
Topic 0: 
 briefing coronavirus happen today test late update
Topic 1: 
 face test mask bad suicide threat stress
Topic 2: 
 test people antibody pandemic f.d.a coronavirus positive
Topic 3: 
 coronavirus late update n.y.c new york pandemic
Topic 4: 
 new pandemic york home market city end


# After

In [31]:
token_list=[]
for headline in df_after.headline:
    head=nlp(headline.lower())
    for token in head:
        if(token.is_stop ==False and token.is_punct == False):
            token_list.append(token.lemma_)
vocab_after=list(set(token_list))

In [32]:
bow_after=pd.DataFrame(0.0, index=np.arange(len(df_after)),columns=vocab_after)
idx=0
for headline in df_after.headline:
    head=nlp(headline.lower())
    for token in head:
        if(token.is_stop ==False and token.is_punct == False):
            bow_after.loc[idx,token.lemma_]+=1
    idx+=1   
bow_after.head(5)

,guardia,institution,airborne,gambia,australia,nomination,thriving,barnard,celebration,sommeli,...,career,gary,64,goodbye,farm,increase,falsely,see,copenheaven,mysterious
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
for col in bow_after.columns:
    if(col.isdigit() or col in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday'] ):
        bow_after.drop([col],inplace=True, axis=1)
    

In [34]:
idf_after={}
for a in bow_after.columns:
    idf_after[a]=np.log(len(bow_after)/(len(bow_after[bow_after[a]!=0])+1))
print(idf_after)

{'guardia': 7.08799132928507, 'institution': 6.682526221176906, 'airborne': 7.08799132928507, 'gambia': 7.08799132928507, 'australia': 7.08799132928507, 'nomination': 7.08799132928507, 'thriving': 7.08799132928507, 'barnard': 6.682526221176906, 'celebration': 7.08799132928507, 'sommeli': 7.08799132928507, 'office': 5.383243237046645, 'adida': 7.08799132928507, 'idea': 6.171700597410915, 'interior': 6.682526221176906, 'approve': 6.171700597410915, 'rebellion': 7.08799132928507, 'mosque': 7.08799132928507, 'succeed': 7.08799132928507, 'bill': 5.2962318600570155, 'tournament': 6.682526221176906, 'application': 6.682526221176906, 'automated': 7.08799132928507, 'staff': 6.394844148725125, 'brutalize': 7.08799132928507, 'nebraska': 7.08799132928507, 'potter': 7.08799132928507, 'scenario': 7.08799132928507, 'employer': 7.08799132928507, 'skin': 7.08799132928507, 'flower': 7.08799132928507, 'testimony': 7.08799132928507, 'take': 4.690096056486699, 'bizarre': 7.08799132928507, 'apparent': 7.087

In [35]:
tfidf_after=bow_after.copy()
tfidf_after["containing_term"]=tfidf_after.gt(0).sum(axis=1)
print(len(tfidf_after[tfidf_after["containing_term"]==0]))
print(tfidf_after[tfidf_after["containing_term"]==0])
tfidf_after=tfidf_after.iloc[:,:].div(tfidf_after.containing_term,axis=0)
for key in idf_after.keys():
    tfidf_after.loc[:,key]=tfidf_after.loc[:,key]*idf_after[key]
tfidf_after.drop(["containing_term"],axis=1, inplace=True)
tfidf_after.dropna(how='any',axis=0,inplace=True)

0
Empty DataFrame
Columns: [guardia, institution, airborne, gambia, australia, nomination, thriving, barnard, celebration, sommeli, office, adida, idea, interior, approve, rebellion, mosque, succeed, bill, tournament, application, automated, staff, brutalize, nebraska, potter, scenario, employer, skin, flower, testimony, take, bizarre, apparent, disadvantage, lake, player, horrifying, buffalo, heyday, east, publish, waive, reimagine, professor, commissioner, tara, distancing, wave, place, french, swimming, harden, meal, devastating, injury, yard, martial, crew, respite, diner, puerto, bangkok, failure, stab, hole, cemetery, w.t.o, v.p, breathtaking, benchmark, autopsy, royal, instrument, reparation, tony, persistence, paradox, stir, cbs, 47th, mile, abolish, ring, superspreading, listener, miracle, livestream, cocktail, nadal, turnabout, scour, react, box, albert, august, serve, fanta, hockey, landlord, ...]
Index: []

[0 rows x 3883 columns]


In [36]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(tfidf_after)
print(svd.explained_variance_)
terms = tfidf_after.columns

for i, comp in enumerate(svd.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    topic=""
    for t in sorted_terms:
        topic=topic+" "+t[0]
    print(topic)

[0.052288   0.04588446 0.016677   0.02613082 0.02434079]
Topic 0: 
 briefing happen today coronavirus evening protest george
Topic 1: 
 result primary election district house california texas
Topic 2: 
 police protest trump new america coronavirus floyd
Topic 3: 
 briefing police protest floyd george trump definitely
Topic 4: 
 trump twitter president rally tweet campaign biden
